In [ ]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import datasets
import util.misc as utils
from datasets import build_matterport_dataset
from models import build_model
from config import cfg


In [ ]:
cmap = plt.get_cmap("jet")
norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

In [ ]:
def c(x):0.2 M 3.0 86.3 90.3 15.8 24.6 36.0 61.7 73.6 84.4
NeurVPS [69] 22 M 0.5 93.9 96.3 24.0 41.8 64.4 52.4 64.0 77.8
Ours 7 M 5.5 84.0 90.2 24.8 42.1 63.7 60.7 74.3 86.3
Ours∗ 5 M 23.0 84.8 90.7 22.9 39.8 62.4 59.5 72.6 85.4
Ours† 7 M 5.5 81.7 88.7 22.2 38.8 59.9 59.1 72.6
                        help="path to config file",
                        type=str,
                        default='/home/kmuvcl/source/CTRL-C/config-files/ctrl-c.yaml')
    parser.add_argument("--opts",
                        help="Modify config options using the command-line",
                        default=None,
                        nargs=argparse.REMAINDER
                        )
    return parser

def compute_vp_err(vp1, vp2, dim=-1):
    cos_sim = F.cosine_similarity(vp1, vp2, dim=dim).abs()
    cos_sim = np.clip(cos_sim.item(), 0.0, 1.0)    
    return np.degrees(np.arccos(cos_sim))

def compute_hl_np(hl, sz, eps=1e-6):
    (a,b,c) = hl
    if b < 0:
        a, b, c = -a, -b, -c
    b = np.maximum(b, eps)
    
    left = np.array([-1.0, (a - c)/b])        
    right = np.array([1.0, (-a - c)/b])

    # scale back to original image    
    scale = sz[1]/2
    left = scale*left
    right = scale*right
    return [np.squeeze(left), np.squeeze(right)]

def compute_up_vector(zvp, fovy, eps=1e-7):
    # image size 2 (-1~1)
    focal = 1.0/np.tan(fovy/2.0)
    
    if zvp[2] < 0:
        zvp = -zvp
    zvp = zvp / np.maximum(zvp[2], eps)
    zvp[2] = focal
    return normalize_safe_np(zvp)

def decompose_up_vector(v):
    pitch = np.arcsin(v[2])
    roll = np.arctan(-v[0]/v[1])
    return pitch, roll

def cosine_similarity(v1, v2, eps=1e-7):
    v1 = v1 / np.maximum(LA.norm(v1), eps)
    v2 = v2 / np.maximum(LA.norm(v2), eps)
    return np.sum(v1*v2)

def normalize_safe_np(v, eps=1e-7):
    return v/np.maximum(LA.norm(v), eps)

def compute_up_vector_error(pred_zvp, pred_fovy, target_up_vector):
    pred_up_vector = compute_up_vector(pred_zvp, pred_fovy)
    cos_sim = cosine_similarity(target_up_vector, pred_up_vector)

    target_pitch, target_roll = decompose_up_vector(target_up_vector)

    if cos_sim < 0:
        pred_pitch, pred_roll = decompose_up_vector(-pred_up_vector)
    else:
        pred_pitch, pred_roll = decompose_up_vector(pred_up_vector)

    err_angle = np.degrees(np.arccos(np.abs(cos_sim)))
    err_pitch = np.degrees(np.abs(pred_pitch - target_pitch))
    err_roll = np.degrees(np.abs(pred_roll - target_roll))
    return err_angle, err_pitch, err_roll

def compute_fovy_error(pred_fovy, target_fovy):
    pred_fovy = np.degrees(pred_fovy)
    target_fovy = np.degrees(target_fovy)
    err_fovy = np.abs(pred_fovy - target_fovy)
    return err_fovy.item()

def compute_horizon_error(pred_hl, target_hl, img_sz):
    target_hl_pts = compute_hl_np(target_hl, img_sz)
    pred_hl_pts = compute_hl_np(pred_hl, img_sz)
    err_hl = np.maximum(np.abs(target_hl_pts[0][1] - pred_hl_pts[0][1]),
                        np.abs(target_hl_pts[1][1] - pred_hl_pts[1][1]))
    err_hl /= img_sz[0] # height
    return err_hl
    
def draw_attention(img, weights, cmap, savepath):
    extent = [-1, 1, 1, -1]
    num_layer = len(weights)
    plt.figure(figsize=(num_layer*3,3))
    for idx_l in range(num_layer):                    
        plt.subplot(1, num_layer, idx_l + 1)
        plt.imshow(img, extent=extent)
        plt.imshow(weights[idx_l], cmap=cmap, alpha=0.3, extent=extent)
        plt.axis('off')
    plt.savefig(savepath, pad_inches=0, bbox_inches='tight')
    plt.close('all')

def draw_attention_segs(img, weights, segs, cmap, savepath):
    num_layer = len(weights)
    num_segs = len(segs)
    plt.figure(figsize=(num_layer*3,3))
    for idx_l in range(num_layer):                    
        plt.subplot(1, num_layer, idx_l + 1)
        plt.imshow(img, extent=[-1, 1, 1, -1])                 
        ws = weights[idx_l]
        ws = (ws - ws.min())/(ws.max() - ws.min())
        for idx_s in range(num_segs):
            plt.plot((segs[idx_s,0], segs[idx_s,2]), 
                     (segs[idx_s,1], segs[idx_s,3]), c=cmap(ws[idx_s]))
        plt.axis('off')
    plt.savefig(savepath, pad_inches=0, bbox_inches='tight')
    plt.close('all')    
    
def to_device(data, device):
    if type(data) == dict:
        return {k: v.to(device) for k, v in data.items()}
    return [{k: v.to(device) if isinstance(v, torch.Tensor) else v
             for k, v in t.items()} for t in data]

In [ ]:
device = torch.device(cfg.DEVICE)
model, _ = build_model(cfg)
model.to(device)

dataset_test = build_matterport_dataset(image_set='train', cfg=cfg)
sampler_test = torch.utils.data.SequentialSampler(dataset_test)
data_loader_test = DataLoader(dataset_test, 1, sampler=sampler_test,
                                drop_last=False, 
                                collate_fn=utils.collate_fn, 
                                num_workers=2)

output_dir = Path(cfg.OUTPUT_DIR)

checkpoint = torch.load('/home/kmuvcl/source/CTRL-C/line_embed/checkpoint0099.pth', map_location='cpu')
model.load_state_dict(checkpoint['model'])
model = model.eval()

# initlaize for visualization
name = f'gsv_test_{date.today()}'
if cfg.TEST.DISPLAY:
    fig_output_dir = osp.join(output_dir,'{}'.format(name))
    os.makedirs(fig_output_dir, exist_ok=True)

dict = {'vp_dot_product':[]}

In [ ]:
for i, (samples, extra_samples, targets) in enumerate(tqdm(data_loader_test)):
        with torch.no_grad():
            samples = samples.to(device)
            print(type(extra_samples))
            extra_samples = to_device(extra_samples, device)
            outputs, extra_info = model(samples, extra_samples)
            img = targets[0]['org_img']
        
            pred_vp1 = outputs['pred_vp1'].to('cpu')[0].numpy()
            pred_vp2 = outputs['pred_vp2'].to('cpu')[0].numpy()
            pred_vp3 = outputs['pred_vp3'].to('cpu')[0].numpy()

            print(pred_vp1,pred_vp2,pred_vp3)

In [1]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv
import warnings

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch

import datasets
import util.misc as utils
from datasets import build_matterport_dataset
from models import build_model
from config import cfg

/home/kmuvcl/anaconda3/envs/debug/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_device(data, device):
    if type(data) == dict:
        return {k: v.to(device) for k, v in data.items()}
    return [{k: v.to(device) if isinstance(v, torch.Tensor) else v
             for k, v in t.items()} for t in data] 

In [3]:
model, _ = build_model(cfg)
dataset_test = build_matterport_dataset(image_set='test', cfg=cfg)

In [4]:
dataset_test = build_matterport_dataset(image_set='test', cfg=cfg)
sampler_test = torch.utils.data.SequentialSampler(dataset_test)
data_loader_test = DataLoader(dataset_test, 1, sampler=sampler_test,
                                drop_last=False, 
                                collate_fn=utils.collate_fn, 
                                num_workers=2)
device = torch.device(cfg.DEVICE)

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

In [5]:
for i, (samples, extra_samples, targets) in enumerate(tqdm(data_loader_test)):
    with torch.no_grad():
        samples = samples.to(device)
        extra_samples = to_device(extra_samples, device)
        samples = samples
        extra_samples = extra_samples
        break

  0%|          | 0/4180 [00:05<?, ?it/s]


In [6]:
samples

tensor([[[[-0.1264, -0.1780,  0.8407,  ...,  1.0735,  0.2685, -1.8008],
          [ 0.0891, -0.8786,  0.1789,  ...,  1.2273,  1.7090, -0.3862],
          [-1.3127,  1.2338, -0.5457,  ..., -0.5208,  2.7156,  0.4870],
          ...,
          [-0.3699, -1.0791,  1.0718,  ...,  0.9229,  0.2116,  0.4318],
          [ 2.1544, -0.7103, -0.0838,  ..., -0.6311,  1.7091,  0.7185],
          [-0.2634,  0.5566,  0.1837,  ...,  1.1263,  1.2414,  1.8262]],

         [[ 1.5707, -0.1305, -2.1983,  ..., -0.6357, -0.7531, -0.2143],
          [ 0.9331,  0.9583, -2.4119,  ...,  0.4475, -0.2008, -0.7467],
          [ 1.0358, -2.0862, -1.5219,  ..., -0.4352, -0.4937, -1.1182],
          ...,
          [ 0.3479,  1.6038, -0.6712,  ...,  0.1415, -0.7793, -1.4379],
          [-0.1185,  0.1389, -0.3428,  ..., -1.3169, -0.4155,  0.8063],
          [-1.9981, -0.5572, -2.4026,  ...,  1.8289,  0.9017, -0.1515]],

         [[-0.1984,  0.0052, -0.0089,  ...,  0.2055, -0.7555, -0.0283],
          [ 0.4654,  0.0470, -

In [ ]:
extra_samples['lines']

In [ ]:
summary(model,(samples,extra_samples))

In [7]:
params = list(model.parameters())

In [8]:
for i, param in enumerate(params):
    print(f"Parameter {i}: {param.shape}")

Parameter 0: torch.Size([768, 256])
Parameter 1: torch.Size([768])
Parameter 2: torch.Size([256, 256])
Parameter 3: torch.Size([256])
Parameter 4: torch.Size([2048, 256])
Parameter 5: torch.Size([2048])
Parameter 6: torch.Size([256, 2048])
Parameter 7: torch.Size([256])
Parameter 8: torch.Size([256])
Parameter 9: torch.Size([256])
Parameter 10: torch.Size([256])
Parameter 11: torch.Size([256])
Parameter 12: torch.Size([768, 256])
Parameter 13: torch.Size([768])
Parameter 14: torch.Size([256, 256])
Parameter 15: torch.Size([256])
Parameter 16: torch.Size([2048, 256])
Parameter 17: torch.Size([2048])
Parameter 18: torch.Size([256, 2048])
Parameter 19: torch.Size([256])
Parameter 20: torch.Size([256])
Parameter 21: torch.Size([256])
Parameter 22: torch.Size([256])
Parameter 23: torch.Size([256])
Parameter 24: torch.Size([768, 256])
Parameter 25: torch.Size([768])
Parameter 26: torch.Size([256, 256])
Parameter 27: torch.Size([256])
Parameter 28: torch.Size([2048, 256])
Parameter 29: torch.

In [9]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
print(f"총 학습 가능한 파라미터 수: {total_params}")

총 학습 가능한 파라미터 수: 7896076


31653132